In [1]:
!git clone https://github.com/llm-fine-tuning/LLaMA-Factory.git

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 12808, done.
remote: Total 12808 (delta 0), reused 0 (delta 0), pack-reused 12808 (from 1)
Receiving objects: 100% (12808/12808), 229.09 MiB | 11.27 MiB/s, done.
Resolving deltas: 100% (9425/9425), done.
Updating files: 100% (227/227), done.


In [2]:
%cd LLaMA-Factory

/workspace/LLaMA-Factory


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 958.4 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.8/170.8 kB 4.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 177.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.7/164.7 kB 314.7 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 584.2 kB/s eta 0:00:00a 0:00:01
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible

In [9]:
!pip install deepspeed==0.14.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for deepspeed: filename=deepspeed-0.14.4-py3-none-any.whl size=1445507 sha256=0df9a4055ff57d38750ad88dec14213dd9c04fad76b4c1fc988da33903e8fdc1
  Stored in directory: /root/.cache/pip/wheels/8e/bc/a3/608e90bbb301848b78fd75d24d6d43ba3074de968fc0e397ac
Successfully built deepspeed
  Attempting uninstall: deepspeed
    Found existing installation: deepspeed 0.14.0
    Uninstalling deepspeed-0.14.0:
      Successfully uninstalled deepspeed-0.14.0

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


## train_sft.sh

deepspeed --num_gpus 6 --master_port=9901 src/train.py \
	--deepspeed ds_z3_config.json \
    --stage sft \
    --do_train \
    --model_name_or_path allganize/Llama-3-Alpha-Ko-8B-Instruct \
    --dataset esg_binary_classification \
    --template llama3-esg-binary-classification \
    --finetuning_type lora \
    --lora_target all \
    --output_dir checkpoint \
    --overwrite_cache \
    --per_device_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --lr_scheduler_type cosine \
    --logging_steps 10 \
    --save_steps 100 \
    --learning_rate 1e-4 \
    --num_train_epochs 10.0 \
    --report_to none \
    --cutoff_len 2000
    --bf16

In [34]:
!python merge.py \
    --base_model_name_or_path allganize/Llama-3-Alpha-Ko-8B-Instruct \
    --peft_model_path ./checkpoint/checkpoint-140 \
    --output_dir ./output_dir

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading base model: allganize/Llama-3-Alpha-Ko-8B-Instruct
Loading checkpoint shards: 100%|██████████████████| 4/4 [00:06<00:00,  1.56s/it]
Loading PEFT: ./checkpoint/checkpoint-160
Running merge_and_unload
[2024-12-03 13:31:49,690] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.4
 [WARNING]  using untested triton version (3.0.0), only 1.0.0 is known to be compatible
/usr/local/lib/python3.10/dist-packages/deepspeed/runtime/zero/linear.py:49: FutureWarning: `torch

In [11]:
from huggingface_hub import HfApi
api = HfApi()
username = "iamjoon"

In [36]:
MODEL_NAME = 'Llama-3-ESG-BINARY-CLASSIFIER'

In [37]:
api.create_repo(
    token="hf_키값",
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)

api.upload_folder(
    token="hf_키값",
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="output_dir_2",
)

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/iamjoon/Llama-3-ESG-BINARY-CLASSIFIER-2/commit/b0f6439be7d8f60ced6f5d27101bc5c87e7c7a16', commit_message='Upload folder using huggingface_hub', commit_description='', oid='b0f6439be7d8f60ced6f5d27101bc5c87e7c7a16', pr_url=None, repo_url=RepoUrl('https://huggingface.co/iamjoon/Llama-3-ESG-BINARY-CLASSIFIER-2', endpoint='https://huggingface.co', repo_type='model', repo_id='iamjoon/Llama-3-ESG-BINARY-CLASSIFIER-2'), pr_revision=None, pr_num=None)

In [5]:
!pip install vllm==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.6/170.6 MB 4.2 MB/s eta 0:00:0000:0100:02m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 3.9 MB/s eta 0:00:0000:0100:04m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 2.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 MB 14.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 21.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 13.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 25.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 25.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [14]:
import string
from vllm import LLM, SamplingParams

# LLM
llm = LLM(model="iamjoon/Llama-3-ESG-BINARY-CLASSIFIER")

config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

INFO 12-03 13:21:48 llm_engine.py:213] Initializing an LLM engine (v0.6.0) with config: model='iamjoon/Llama-3-ESG-BINARY-CLASSIFIER', speculative_config=None, tokenizer='iamjoon/Llama-3-ESG-BINARY-CLASSIFIER', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=iamjoon/Llama-3-ESG-BINARY-CLASSIFIER, use_v2_block_manager=False, num_scheduler_steps=1, enable_pr

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 12-03 13:24:00 model_runner.py:926] Loading model weights took 14.9595 GB
INFO 12-03 13:24:01 gpu_executor.py:122] # GPU blocks: 27978, # CPU blocks: 2048
INFO 12-03 13:24:04 model_runner.py:1217] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 12-03 13:24:04 model_runner.py:1221] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 12-03 13:24:14 model_runner.py:1335] Graph capturing finished in 10 secs.


In [38]:
# 프롬프트 템플릿 준비
template = string.Template("""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

${system_content}<|eot_id|><|start_header_id|>user<|end_header_id|>

${user_content}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
""")

system_prompt = '''1. 상장사의 뉴스를 보고 ESG 관련 기사인지 판단하시오.
2. ESG 각각에 대한 핵심 키워드는 아래의 10개 키워드를 참고해서 판단하시오.
- 환경 (Environmental) : 탄소 배출 감소 (Carbon Emission Reduction), 재생 에너지 사용 (Renewable Energy Usage), 자원 효율성 (Resource Efficiency), 폐기물 관리 (Waste Management), 친환경 제품 개발 (Eco-Friendly Product Development), 기후 변화 대응 (Climate Change Mitigation), 오염 방지 (Pollution Prevention), 생물 다양성 보호 (Biodiversity Protection), 물 관리 (Water Management)
순환 경제 (Circular Economy)
- 사회 (Social) : 노동 인권 보호 (Labor Rights Protection), 다양성 및 포용성 (Diversity and Inclusion), 직원 안전 및 건강 (Employee Health & Safety), 공정한 노동 관행 (Fair Labor Practices), 커뮤니티 참여 및 개발 (Community Engagement & Development), 제품 책임 및 안전 (Product Responsibility & Safety), 공급망 책임 (Supply Chain Responsibility), 데이터 보호 및 프라이버시 (Data Protection & Privacy), 인재 개발 및 교육 (Talent Development & Education), 고객 만족도 (Customer Satisfaction)
- 지배구조 (Governance) : 이사회 독립성 (Board Independence), 윤리적 경영 (Ethical Management), 주주 권리 보호 (Shareholder Rights Protection), 내부 통제 및 감사 (Internal Controls & Audits), 리스크 관리 (Risk Management), 투명한 정보 공개 (Transparent Disclosure), 기업 윤리 및 부패 방지 (Corporate Ethics & Anti-Corruption), 경영진 보상 구조 (Executive Compensation Structure), 컴플라이언스 (Regulatory Compliance), 이해관계자 참여 (Stakeholder Engagement)
3. 입력이 주어지면 근거를 작성하고 ESG 관련 기사이면 => True 아니면 => False를 작성합니다.
4. 답변은 항상 '=> True' 또는 '=> False'로 끝나야 하며 그 뒤에 아무 것도 적지마세요.
5. 지역의 정책이나 장소에 대한 내용이거나, 개인의 사건과 같이 특정 기업에 대한 이야기가 아닌 경우에는 아니라고 판단하세요. 저는 상장 '기업'의 내용에 집중하고 있습니다.
6. 기업의 경영권 분쟁과 같은 경우에는 단순히 개인의 이야기라고 볼 수 없습니다. 이 경우에도 ESG라고 판단할 수 있습니다.
7. 개인적인 부고의 기사는 ESG와 관련이 없다고 판단하세요.'''

user_prompt = '''LG에너지솔루션, 배터리 관리 분야 새 얼굴 '비 어라운드' 만들었다 LG에너지솔루션의 배터리 관리 토털 설루션(BMTS) 브랜드 'B.around'(비.어라운드) 로고. LG에너지솔루션 제공  잇단 화재로 전기차 배터리에 불안과 공포(전기차 포비아)가 확산한 가운데 LG에너지솔루션(LG엔솔)이 배터리 관리 토털 설루션(BMTS) 브랜드를 만들고 사업 확장에 나선다. 전기차 배터리 생산을 넘어 소프트웨어 부문 강화로 수익원을 넓히려는 움직임으로 풀이된다.  LG엔솔은 25일 '항상 고객의 곁에'(Be around your side)란 뜻의 BMTS 브랜드 'B.around'(비.어라운드)를 공개했다. 어떤 상황에서든 배터리 상태를 실시간 모니터링하는 LG엔솔 BMTS의 기술 경쟁력을 상징한다는 설명이다.  비.어라운드 제품에는 기존 배터리 관리 시스템(BMS)에 클라우드와 인공지능(AI) 기술을 결합해 화재방지 등을 위한 안전 진단, 퇴화·수명 예측 등을 더 쉽게 할 계획이다. 배터리 불량 유형을 사전에 진단하고 퇴화 상태를 점검하는 소프트웨어를 통해 오랫동안 쓸 수 있게 할 방침이다.  이달훈 LG에너지솔루션 BMS개발센터장 상무는 "오랜 기간 안전 진단 시스템을 위해 노력했고 이제는 배터리 관리 소프트웨어 사업으로 확장해 배터리의 건강한 사용을 위해 설루션을 제공하는 기업이 되려고 한다"고 말했다.  김청환 기자 chk@hankookilbo.com'''

# 메시지 준비
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

# 프롬프트 생성
prompt = template.safe_substitute({
    "system_content": messages[0]["content"],
    "user_content": messages[1]["content"]
})

In [39]:
prompt = prompt.strip() + '근거:'

In [40]:
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

1. 상장사의 뉴스를 보고 ESG 관련 기사인지 판단하시오.
2. ESG 각각에 대한 핵심 키워드는 아래의 10개 키워드를 참고해서 판단하시오.
- 환경 (Environmental) : 탄소 배출 감소 (Carbon Emission Reduction), 재생 에너지 사용 (Renewable Energy Usage), 자원 효율성 (Resource Efficiency), 폐기물 관리 (Waste Management), 친환경 제품 개발 (Eco-Friendly Product Development), 기후 변화 대응 (Climate Change Mitigation), 오염 방지 (Pollution Prevention), 생물 다양성 보호 (Biodiversity Protection), 물 관리 (Water Management)
순환 경제 (Circular Economy)
- 사회 (Social) : 노동 인권 보호 (Labor Rights Protection), 다양성 및 포용성 (Diversity and Inclusion), 직원 안전 및 건강 (Employee Health & Safety), 공정한 노동 관행 (Fair Labor Practices), 커뮤니티 참여 및 개발 (Community Engagement & Development), 제품 책임 및 안전 (Product Responsibility & Safety), 공급망 책임 (Supply Chain Responsibility), 데이터 보호 및 프라이버시 (Data Protection & Privacy), 인재 개발 및 교육 (Talent Development & Education), 고객 만족도 (Customer Satisfaction)
- 지배구조 (Governance) : 이사회 독립성 (Board Independence), 윤리적 경영 (Ethical Management), 주주 권리 

In [41]:
# 샘플링 파라미터 설정
sampling_params = SamplingParams(
    temperature=0,
    max_tokens=1024,
    stop=["\n<|end_of_text|>", "<|eot_id|>"],  # 문자열로 중지 토큰 지정
    repetition_penalty=1.0
)

# 생성
outputs = llm.generate([prompt], sampling_params)

# 결과 출력
for output in outputs:
    print()
    print(output.outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.62s/it, est. speed input: 381.03 toks/s, output: 79.03 toks/s]


 LG에너지솔루션이 배터리 관리 분야에서 새로운 브랜드 'B.around'을 출시하고, 소프트웨어 부문 강화를 통해 수익원을 넓히려는 움직임은 ESG와 관련이 있습니다. 첫째, 환경 측면에서 배터리 관리 기술을 통해 전기차 배터리의 안전성을 높이고 화재 방지 및 퇴화 예측을 통해 자원 효율성을 개선하려는 노력은 환경 보호와 관련이 있습니다. 둘째, 사회적 측면에서는 배터리 관리 소프트웨어를 통해 배터리의 건강한 사용을 촉진하고, 고객에게 안전한 제품을 제공하려는 노력은 제품 책임 및 안전과 관련이 있습니다. 마지막으로 지배구조 측면에서는 기술 혁신을 통해 기업의 책임 있는 경영을 보여주는 사례로, ESG의 지배구조 원칙을 따르는 행보입니다. 
=> True


In [20]:
!wget https://raw.githubusercontent.com/llm-fine-tuning/LLaMA-Factory/refs/heads/main/data/esg_binary_classification_test.json

--2024-12-03 13:26:10--  https://raw.githubusercontent.com/llm-fine-tuning/LLaMA-Factory/refs/heads/main/data/esg_binary_classification_test.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


200 OK
Length: 2728282 (2.6M) [text/plain]
Saving to: ‘esg_binary_classification_test.json’

esg_binary_classifi 100%[===================>]   2.60M  11.3MB/s    in 0.2s    

2024-12-03 13:26:11 (11.3 MB/s) - ‘esg_binary_classification_test.json’ saved [2728282/2728282]



In [21]:
import pandas as pd

df = pd.read_json('esg_binary_classification_test.json')

In [22]:
df

,instruction,input,output
0,"""월남쌈 대신 '비비고' 만두 먹고 CGV서 영화 봐요""…'베트남 젠지' 홀린 CJ...",,근거: 이 기사는 CJ그룹이 베트남에서 개최한 'CJ K FESTA'라는 K컬처 축...
1,"KDI ""한국경제 부진 지속…수출 위축에 내수도 둔화"" 국책연구원인 한국개발연구원,...",,"근거: 이 기사는 국책연구원인 한국개발연구원(KDI)의 한국 경제에 대한 분석으로,..."
2,개성 가득 이색 팝업 ’글로스튜디오’ 체험하는 배우 이이경 (서울=뉴스1) = BA...,,근거: 이 기사는 BAT로스만스의 이색 팝업 '글로 스튜디오' 오픈 행사와 배우 이...
3,"삼성전자 노조, 사측에 화해 손짓…""파업 상황 조속 해결"" 전국삼성전자노동조합 조합...",,근거: 해당 기사는 삼성전자의 노조와 사측 간의 파업 및 대화 제안에 관한 내용으로...
4,"‘클라우드’ 돕는 트레이너…‘AI’까지 넘본다 [천억클럽] 인공지능(AI) 시대, ...",,근거: 이 기사는 주로 클라우드 MSP(Managed Service Provider...
...,...,...,...
927,부산시 전기차 구매 보조금 지원대상 확대 [KBS 부산]부산시가 올해 상반기 전기자...,,근거: 부산시 전기차 구매 보조금 지원 대상 확대는 특정 기업과 관련된 이야기가 아...
928,KT&G “세계적인 의결권자문사의 이사회 제안 안건 ‘찬성 권고’ 대환영” 글로벌 ...,,근거: 해당 기사는 KT&G의 주주총회와 이사회 제안에 대한 의결권 자문사의 권고와...
929,서산 농작물 창고서 불…2백만 원 피해 [KBS 대전]오늘(27일) 오전 10시쯤 ...,,"근거: 이 기사는 서산의 농작물 창고에서 발생한 화재 사건에 관한 것으로, 특정 기..."
930,"현대차, 신형 코나 전기차 모델 첫 공개 자동요약 기사 제목과 주요 문장을 기반으로...",,근거: 현대차가 코나의 전기차 모델 '디 올 뉴 코나 일렉트릭'을 공개한 것은 ES...


In [23]:
inputs = df['instruction'].to_list()

In [42]:
def make_prompt(system_prompt, news_article):
    # 프롬프트 템플릿 준비
    template = string.Template("""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    
${system_content}<|eot_id|><|start_header_id|>user<|end_header_id|>
    
${user_content}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
""")
    
    user_prompt = news_article
    
    # 메시지 준비
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    # 프롬프트 생성
    prompt = template.safe_substitute({
        "system_content": messages[0]["content"],
        "user_content": messages[1]["content"]
    })

    prompt = prompt.strip() + "근거:"
    return prompt

In [43]:
prompt = make_prompt(system_prompt, inputs[0])
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    
1. 상장사의 뉴스를 보고 ESG 관련 기사인지 판단하시오.
2. ESG 각각에 대한 핵심 키워드는 아래의 10개 키워드를 참고해서 판단하시오.
- 환경 (Environmental) : 탄소 배출 감소 (Carbon Emission Reduction), 재생 에너지 사용 (Renewable Energy Usage), 자원 효율성 (Resource Efficiency), 폐기물 관리 (Waste Management), 친환경 제품 개발 (Eco-Friendly Product Development), 기후 변화 대응 (Climate Change Mitigation), 오염 방지 (Pollution Prevention), 생물 다양성 보호 (Biodiversity Protection), 물 관리 (Water Management)
순환 경제 (Circular Economy)
- 사회 (Social) : 노동 인권 보호 (Labor Rights Protection), 다양성 및 포용성 (Diversity and Inclusion), 직원 안전 및 건강 (Employee Health & Safety), 공정한 노동 관행 (Fair Labor Practices), 커뮤니티 참여 및 개발 (Community Engagement & Development), 제품 책임 및 안전 (Product Responsibility & Safety), 공급망 책임 (Supply Chain Responsibility), 데이터 보호 및 프라이버시 (Data Protection & Privacy), 인재 개발 및 교육 (Talent Development & Education), 고객 만족도 (Customer Satisfaction)
- 지배구조 (Governance) : 이사회 독립성 (Board Independence), 윤리적 경영 (Ethical Management), 주주

In [44]:
prompts = []
for input in inputs:
    prompts.append(make_prompt(system_prompt, input))

## 다수의 데이터 호출

In [45]:
results = llm.generate(prompts[:5], sampling_params)

Processed prompts: 100%|██████████| 5/5 [00:02<00:00,  2.16it/s, est. speed input: 3656.48 toks/s, output: 200.21 toks/s]


In [46]:
for result in results:
    print(result.outputs[0].text)
    print('--')

 이 기사는 CJ그룹이 베트남에서 'CJ K FESTA'라는 문화 축제를 개최하고 브랜드 홍보를 강화하는 내용으로, ESG와 직접적인 관련이 없습니다. 이 기사는 기업의 마케팅 및 브랜드 전략에 관한 내용으로, 환경, 사회, 지배구조와 관련된 ESG 요소와는 연관이 없습니다. 
=> False
--
 이 기사는 한국개발연구원(KDI)이 한국 경제의 부진을 진단한 내용으로, 특정 기업에 대한 이야기가 아닙니다. 따라서 ESG와 관련이 없습니다. 
=> False
--
 이 기사는 BAT로스만스가 서울 중구에 '글로스튜디오'라는 팝업 체험 공간을 오픈했다는 내용으로, 이는 기업의 마케팅 및 홍보 활동에 관한 이야기입니다. ESG와 관련된 환경, 사회, 지배구조 측면의 핵심 키워드와 직접적인 관련이 없습니다. 따라서 ESG 관련 기사로 판단하기 어렵습니다. 
=> False
--
 이 기사는 삼성전자 노조의 파업과 관련된 내용으로, 이는 ESG의 사회(Social) 측면과 관련이 있습니다. 노동 인권 보호(Labor Rights Protection)와 공정한 노동 관행(Fair Labor Practices)과 관련된 문제가 포함되어 있습니다. 노조의 활동과 임금 교섭, 파업은 노동자들의 권리와 복지를 위한 활동으로, ESG의 사회적 책임과 관련이 있습니다. 
=> True
--
 이 기사는 베스핀글로벌이라는 기업의 클라우드 서비스와 AI 기술에 대한 내용을 다루고 있습니다. ESG와 관련된 키워드 중 환경, 사회, 지배구조 측면에서 직접적인 관련성을 찾기 어렵습니다. 클라우드 서비스와 AI 기술은 기업의 운영 효율성과 관련이 있을 수 있지만, 기사에서 명확하게 ESG 원칙과 연결되는 부분은 언급되지 않았습니다. 따라서 이 기사는 ESG 관련 기사로 판단하기 어렵습니다. 
=> False
--


In [47]:
results = llm.generate(prompts, sampling_params)

Processed prompts: 100%|██████████| 932/932 [01:44<00:00,  8.91it/s, est. speed input: 10958.67 toks/s, output: 982.79 toks/s]


In [57]:
labels = []
for label in df['output'].to_list():
    labels.append(label.split('=>')[1].strip())

preds = []
for pred in results:
    preds.append(pred.outputs[0].text.split('=>')[1].strip())

In [58]:
labels[:5]

['False', 'False', 'False', 'True', 'False']

In [59]:
preds[:5]

['False', 'False', 'False', 'True', 'False']

In [63]:
def calculate_metrics(labels, preds):
    """
    정확도와 F1 점수를 계산하는 함수 (문자열 "True"/"False" 사용).
    
    Args:
        labels (list): 실제 값 리스트 ("True"/"False").
        preds (list): 예측 값 리스트 ("True"/"False").

    Returns:
        dict: 정확도와 F1 점수를 포함한 결과.
    """
    # True Positive, False Positive, False Negative, True Negative 계산
    tp = sum(1 for l, p in zip(labels, preds) if l == "True" and p == "True")
    fp = sum(1 for l, p in zip(labels, preds) if l == "False" and p == "True")
    fn = sum(1 for l, p in zip(labels, preds) if l == "True" and p == "False")
    tn = sum(1 for l, p in zip(labels, preds) if l == "False" and p == "False")
    
    # Precision, Recall 계산
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    # F1 Score 계산
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # Accuracy 계산
    accuracy = (tp + tn) / len(labels) if len(labels) > 0 else 0
    
    return {"accuracy": accuracy, "f1_score": f1_score}

# 함수 호출
metrics = calculate_metrics(labels, preds)
print(metrics)

{'accuracy': 0.8927038626609443, 'f1_score': 0.9}


## 정답과 예측이 같은 경우

In [72]:
df['label'] = labels
df['pred'] = preds

In [71]:
pred_reasons = ['근거:' + r.outputs[0].text for r in results]
df['pred_reasons'] = pred_reasons 

In [83]:
idx = 0

print('뉴스 기사')
print(df['instruction'].loc[idx])
print('--' * 20)
print('정답')
print(df['output'].loc[idx])
print('--' * 20)
print('예측')
print(df['pred_reasons'].loc[idx])

뉴스 기사
"월남쌈 대신 '비비고' 만두 먹고 CGV서 영화 봐요"…'베트남 젠지' 홀린 CJ CJ그룹은 지난 10일부터 9월 1일까지 베트남에서 K컬처 축제 'CJ K FESTA'를 연다.ⓒ 뉴스1/김진희 기자.  (호찌민=뉴스1) 김진희 기자 = "CJ(001040) 로고가 화려하고 예쁩니다. CGV를 통해 CJ를 알게 됐는데 젊고 역동적이어서 '젠지' 세대에 적합한 이미지라는 생각이 듭니다." -화이빠오(24·남)  지난 15~16일(현지시간) 베트남 호찌민 일대에서 열린 'CJ K FESTA'를 찾은 베트남 현지인들은 극찬을 쏟아냈다.  CJ그룹은 베트남에 진출한 계열사를 총동원해 CJ K FESTA를 올해 처음으로 개최했다. 이번 축제는 △K푸드 Week(8월 10~16일) △K스포츠 Week(8월 17~23일) △K무비 Week(8월 24일~9월 1일) 등 3가지 테마로 구성됐다.  CJ는 호찌민 소재 유명 마트와 쇼핑몰 내 CGV 매장에서 푸드쇼를 운영하며 브랜드 홍보에 적극 나섰다.  CJ그룹은 지난 10일부터 9월 1일까지 베트남에서 K컬처 축제 'CJ K FESTA'를 연다.ⓒ 뉴스1/김진희 기자.  15일 오후 호찌민 소재 메가마켓에는 더운 날씨임에도 마트 외부 공간에 사람들로 북적였다. CJ 제품을 선보이는 푸드트럭을 구경하기 위해 찾은 이들이었다. 한국과 달리 베트남에서는 푸드트럭이 생소하다고 한다.  이날 CJ는 CJ제일제당 비비고의 해산물 만두, 호떡, 김스낵과 뚜레쥬르의 빵을 선보였다. CJ제일제당의 자회사인 베트남 현지 냉동식품기업 '까우제'(Cautre)의 제품도 만나볼 수 있었다.  호찌민 시민 화이빠오는 "예전에 만두를 많이 먹어 봤지만 비비고 만두는 독특하고 맛있다"며 "특히 푸드트럭을 처음 봐서 신기한데, 이렇게 화려하게 시식·광고하는 건 처음 본다"고 말했다.  CJ그룹은 지난 10일부터 9월 1일까지 베트남에서 K컬처 축제 'CJ K FESTA'를 연다.ⓒ 뉴스1/김진희 기자.  아이와 함께 푸드트럭을 방문한 주부 흐

## 정답과 예측이 다른 경우

In [73]:
df[df['label']!=df['pred']]

,instruction,input,output,label,pred,pred_reasons
16,"""한 달 빨라졌다""...한화그룹 14개 계열사 정기 임원 인사, 51명 임원 승진 ...",,"근거: 이 기사는 한화그룹의 14개 계열사 정기 임원 인사에 관한 내용으로, 인사가...",True,False,"근거: 이 기사는 한화그룹의 14개 계열사 정기 임원 인사에 대한 내용으로, ESG..."
34,"원유석 아시아나항공 대표 직무대행, 부사장 승진 아시아나항공은 15일 원유석 대표이...",,근거: 아시아나항공의 임원 인사 단행은 ESG의 지배구조(Governance)와 관...,True,False,근거: 아시아나항공의 임원 인사는 ESG와 직접적인 관련이 없습니다. 이는 기업의 ...
48,카카오 자회사 디케이테크인·카카오브레인 합병한다 자동요약 기사 제목과 주요 문장을 ...,,"근거: 이 기사는 카카오 자회사 디케이테크인과 카카오브레인의 합병에 관한 내용으로,...",True,False,근거: 카카오의 자회사인 디케이테크인과 카카오브레인의 합병은 ESG와 직접적인 관련...
61,세계 1위 인구 대국 인도가 꼽은 ‘올해의 차’는 기아 ‘카렌스’ 기아 미니밴 ‘카...,,근거: 이 기사는 기아자동차의 제품이 인도에서 '올해의 차'와 '그린카' 부문에서 ...,False,True,근거: 기아가 인도에서 '올해의 차'와 '그린카' 부문을 수상한 것은 ESG와 관련...
63,폭염에 잘나가던 빙그레…오너 3세 폭행으로 ‘얼음’ [재계 TALK TALK] 역대...,,"근거: 이 기사는 빙그레의 오너 3세인 김동환 사장의 폭행 사건과 관련된 내용으로,...",False,True,근거: 이 기사는 빙그레의 오너 3세인 김동환 사장이 술에 취해 경찰을 폭행한 사건...
...,...,...,...,...,...,...
825,"금융결제원-NICE페이먼츠, 구독 결제 서비스 MOU 정대성 금융결제원 금융사업본부...",,근거: 금융결제원과 NICE페이먼츠의 협력이 구독 결제 서비스 인프라를 구축하는 내...,False,True,근거: 금융결제원과 NICE페이먼츠의 협약은 ESG와 관련이 있습니다. 사회적 측면...
828,"현대차·제네시스, 美 J.D.파워 만족도 조사 1위 [KBS 울산]현대차와 제네시스...",,근거: 현대차와 제네시스의 신차 첨단 기술 만족도 조사 결과가 1위를 차지한 것은 ...,True,False,근거: 현대차와 제네시스가 미국의 J.D.파워 신차 첨단 기술 만족도 조사에서 1위...
842,"“테슬라의 강력한 라이벌”...현대차, 해외에서 ‘극찬’ 서울 양재동 현대차 본사 ...",,근거: 현대차의 전기차 세단 '아이오닉6'가 테슬라 '모델3'의 강력한 라이벌로 평...,True,False,근거: 이 기사는 현대차의 전기차 '아이오닉6'가 테슬라 '모델3'의 강력한 라이벌...
907,"'삐약이' 신유빈, 바나나맛 우유 광고 찍고 '빙그레' 탁구 국가대표 선수 신유빈이...",,"근거: 이 기사는 기업 빙그레와 관련된 내용으로, 신유빈 선수가 출연한 바나나맛우유...",True,False,"근거: 이 기사는 빙그레와 신유빈의 광고 촬영 및 기부 활동에 관한 내용으로, ES..."


In [84]:
idx = 16

print('뉴스 기사')
print(df['instruction'].loc[idx])
print('--' * 20)
print('정답')
print(df['output'].loc[idx])
print('--' * 20)
print('예측')
print(df['pred_reasons'].loc[idx])

뉴스 기사
"한 달 빨라졌다"...한화그룹 14개 계열사 정기 임원 인사, 51명 임원 승진 한화그룹 기업 이미지(CI). 한화그룹 제공  한화그룹이 14개 계열사의 정기 임원 인사를 단행했다. 인사 시기가 예년보다 한 달가량 앞당겨진 것으로 글로벌 경영 환경이 급변하는 데 빠르게 대응하려는 의도란 설명이다.  한화그룹은 27일 지주회사 격인 ㈜한화를 비롯해 총 14개사의 정기 임원 인사를 단행했다고 밝혔다. 한화에어로스페이스, 한화솔루션, 한화오션, 한화시스템, 한화에너지, 한화임팩트, 한화토탈에너지스, 한화파워시스템, 한화엔진, 한화첨단소재, 한화이센셜, 한화비전, 한화정밀기계 등이다.  이번 인사로 총 51명이 임원으로 승진했다. 승진 일자는 10월 1일이다.  한화그룹의 모태이자 주력 사업인 방위 산업을 담당하는 계열사 중 한화에어로스페이스가 가장 많은 12명의 임원을 배출했다. 한화시스템도 3명을 임원으로 승진시켰다. 한화솔루션은 7명의 신임 임원을 발탁했다. 한화솔루션은 현재의 경영 위기를 극복하고 사업 전환을 가속하기 위해 대표이사 인사에 이어 임원 인사도 앞당겨진 것이라고 설명했다.  조선업 부문인 한화오션(옛 대우조선해양)도 이날 임원 7명 승진 인사를 단행했다. 한화오션은 공정 안정화와 차별화된 원가 경쟁력 확보를 위해 현장 경험과 전문성을 겸비한 생산 분야 전문 인력 발탁에 이번 인사의 중점을 뒀다고 밝혔다. 임원 승진자 7명의 소속은 연구개발(R&D) 1명, 설계 2명, 생산 4명 등이다.  이 밖에 한화에너지, 한화임팩트, 한화토탈에너지스, 한화파워시스템, 한화엔진, 한화비전, 한화정밀기계 등 7개사가 총 13명을 신규 임원으로 승진시켰다. 한화솔루션의 자회사인 한화첨단소재와 한화이센셜도 각각 한 명의 임원 승진 인사를 발표했다.  이날 인사가 난 회사는 대부분 김승연 회장을 잇는 한화그룹 오너 3세 삼형제(김동관 김동원 김동선) 가운데 장남 김동관 부회장이 이끄는 곳이란 공통점이 있다. 예년보다 빨라진 인사에서 그의 경영 스타일을 엿볼